In [37]:
import os
path = "bbc" #Folder directory
all_files= os.listdir(path)
all_data = []
class_lable = -1
for sub_file in all_files:
   
    sub_position = path+ '/' + sub_file
    sub_all_files = os.listdir(sub_position)
#     Every_subfile = []
    class_lable += 1
    sum_num = 0
    for file in sub_all_files:
        position = sub_position+ '/' + file
        sum_num += 1
        
        with open(position, "r",encoding='unicode_escape') as f:
            lines = f.readlines()
            for line in lines:
                if line == '\n':
                    line = line.strip()
                else:
                    all_data.append((line.strip(),class_lable))
    print(sub_file,sum_num,' files')
#     all_data.append(Every_subfile)             
        

business 510  files
entertainment 386  files
politics 417  files
sport 511  files
tech 401  files


In [45]:
print(len(all_data))

12893


In [38]:
from sklearn.model_selection import train_test_split
Data_Train, Data_Test = train_test_split(all_data, test_size = 0.2, random_state = 0)
Data_New_Test, Data_New_Dev = train_test_split(Data_Test, test_size = 0.5, random_state = 0)

In [39]:
import random 
Data_New_Train = Data_Train
random.shuffle(Data_New_Train)
random.shuffle(Data_New_Test)
random.shuffle(Data_New_Dev)
print(len(Data_New_Train))
print(len(Data_New_Test))
print(len(Data_New_Dev))

10314
1289
1290


In [50]:
for line in Data_New_Train[:2]:
    print(line)
print("\n------\n")
for line in Data_New_Test[:2]:
    print(line)
print("\n------\n")
for line in Data_New_Dev[:2]:
    print(line)
print("\n------\n")

('"For any cultural output to thrive there needs to be some kind of state input to that as well," he said. But Kapranos warned against musicians being too closely linked with MPs, at the University of Edinburgh event. "I think the role of musicians is to question politicians rather than to go to bed with them," he said.', 1)
("The Japanese gaming giant hopes the DS will maintain the firm's pre-eminence in an increasingly-competitive mobile gaming market.", 4)

------

("At least 10 countries have been affected, with Sri Lanka, Indonesia, India and Thailand among the worst hit. The region's resorts and Western tourists are expected to be among the main claimants.", 0)
("LG Card had been threatened with collapse because of its huge debts but the firm's creditors and its former parent have stepped in to rescue it. A consortium of creditors and LG Group, a family owned conglomerate, have each put up $480m to stabilise the firm. LG Card has seven million customers and its collapse would hav

In [51]:
import numpy as np
import nltk
import sklearn
import operator

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

lemmatizer = nltk.stem.WordNetLemmatizer()
stopwords=set(nltk.corpus.stopwords.words('english'))
stopwords.add(".")
stopwords.add(",")
stopwords.add("--")
stopwords.add("``")
stopwords.add("''")
stopwords.add("&")
stopwords.add("?")
stopwords.add("$")
for ch in "''-)(%:":
    stopwords.add(ch)

# Function taken from Session 1
def get_list_tokens(string): # Function to retrieve the list of tokens from a string
    sentence_split=nltk.tokenize.sent_tokenize(string)
    list_tokens=[]
    for sentence in sentence_split:
        list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
        for token in list_tokens_sentence:
            list_tokens.append(lemmatizer.lemmatize(token).lower())
    return list_tokens

# Function taken from Session 2
def get_vector_text(list_vocab,string):
    vector_text=np.zeros(len(list_vocab))
    list_tokens_string=get_list_tokens(string)
    for i, word in enumerate(list_vocab):
        if word in list_tokens_string:
            vector_text[i]=list_tokens_string.count(word)
    return vector_text


# Functions slightly modified from Session 2

def get_vocabulary(training_set, num_features):
    dict_word_frequency={}
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            if word in stopwords: 
                continue
            if word not in dict_word_frequency: 
                dict_word_frequency[word]=1
            else: 
                dict_word_frequency[word]+=1
    sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:num_features]
    vocabulary=[]
    for word,frequency in sorted_list:
        vocabulary.append(word)
    return vocabulary


def get_num_vocabulary(training_set, num_features):
    dict_word_num={}
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            sum_num = 0
            if word in stopwords: 
                continue
            else:
                for ch in word:
                    sum_num += 1
                dict_word_num[word] = sum_num   

    sorted_list = sorted(dict_word_num.items(), key=operator.itemgetter(1), reverse=True)[:num_features]
    vocabulary=[]
    for word,frequency in sorted_list:
        vocabulary.append(word)
    return vocabulary




def get_ave_num_vocabulary(training_set, num_features):
    dict_word_num={}
    word_num = 0
    all_word_num = 0
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            word_num += 1
            sum_num = 0
            if word in stopwords: 
                continue
            else:
                for ch in word:
                    sum_num += 1
                    all_word_num += 1
                dict_word_num[word] = sum_num   
    ave_num = round(all_word_num/word_num,0)
    
    vocabulary=[]
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            if word in stopwords: 
                continue
            elif dict_word_num[word] == ave_num:
                if word in vocabulary:
                    continue
                else:
                    vocabulary.append(word)
                    
    size_test=int(num_features)

    list_test_indices=random.sample(range(len(vocabulary)), size_test)
    select_vocabulary=[]

    for i,example in enumerate(vocabulary):
        if i in list_test_indices: 
            select_vocabulary.append(example)
    return select_vocabulary

def train_svm_classifier(training_set, vocabulary): 
    X_train=[]
    Y_train=[]
    for instance in training_set:
        vector_instance=get_vector_text(vocabulary,instance[0])
        X_train.append(vector_instance)
        Y_train.append(instance[1])
  

    svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
    svm_clf.fit(np.asarray(X_train),np.asarray(Y_train))
    return svm_clf

# def feature_selection(training_set,vocabulary, num_features):
#     X_train=[]
#     Y_train=[]
#     for instance in training_set:
#         vector_instance=get_vector_text(vocabulary,instance[0])
#         X_train.append(vector_instance)
#         Y_train.append(instance[1])
    
#     X_new_train = np.asarray(X_train)
#     Y_new_train = np.asarray(Y_train)
    
#     fs_data=SelectKBest(chi2, k=num_features).fit(X_new_train, Y_new_train)
#     X_train_fs = fs_data.transform(X_new_train)
#     #X_train_new = SelectKBest(chi2, k=500).fit_transform(X_train, Y_train)
#     print ("Size original training matrix: "+str(X_new_train.shape))
#     print ("Size new training matrix: "+str(X_train_fs.shape))
    
#     svm_clf_fs=sklearn.svm.SVC(kernel="linear",gamma='auto') # Change the name here, e.g. 'new sentanalysis_svm_clf', and below if you don't want to replace your old classifier.
#     svm_clf_fs.fit(X_train_fs,Y_new_train) #Train the new SVM model. This may take a while.
    
#     return svm_clf_fs

In [41]:
def get_ave_num_vocabulary_nonum(training_set): # Function to retrieve vocabulary
    dict_word_num={}
    word_num = 0
    all_word_num = 0
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            word_num += 1
            sum_num = 0
            if word in stopwords: 
                continue
            else:
                for ch in word:
                    sum_num += 1
                    all_word_num += 1
                dict_word_num[word] = sum_num   
    ave_num = round(all_word_num/word_num,0)
    
    vocabulary=[]
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            if word in stopwords: 
                continue
            elif dict_word_num[word] == ave_num:
                if word in vocabulary:
                    continue
                else:
                    vocabulary.append(word)
                    
    return vocabulary


In [42]:
all_ave_num_vocabulary=get_ave_num_vocabulary_nonum(Data_New_Train)
print(len(all_ave_num_vocabulary))

1377


# Frequency Feature

In [43]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

Y_dev=[]
for instance in Data_New_Dev:
    Y_dev.append(instance[1])
Y_dev_gold=np.asarray(Y_dev)


list_num_features=[500,1000,1500,2000]
best_accuracy_dev=0.0
for num_features in list_num_features:
  # First, we get the vocabulary from the training set and train our svm classifier
    vocabulary=get_vocabulary(Data_New_Train, num_features)  
    svm_clf=train_svm_classifier(Data_New_Train, vocabulary)
  # Then, we transform our dev set into vectors and make the prediction on this set
    X_dev=[]
    for instance in Data_New_Dev:
        vector_instance=get_vector_text(vocabulary,instance[0])
        X_dev.append(vector_instance)
    X_dev=np.asarray(X_dev)
    Y_dev_predictions=svm_clf.predict(X_dev)
  # Finally, we get the accuracy results of the classifier
    accuracy_dev=accuracy_score(Y_dev_gold, Y_dev_predictions)
    print ("Accuracy with "+str(num_features)+": "+str(round(accuracy_dev,3)))
    if accuracy_dev>=best_accuracy_dev:
        best_accuracy_dev=accuracy_dev
        best_num_features=num_features
        frequency_best_vocabulary=vocabulary
        frequency_best_svm_clf=svm_clf
print ("\n Frequency best accuracy overall in the dev set is "+str(round(best_accuracy_dev,3))+" with "+str(best_num_features)+" features.")

Accuracy with 500: 0.819
Accuracy with 1000: 0.846
Accuracy with 1500: 0.863
Accuracy with 2000: 0.875

 Frequency best accuracy overall in the dev set is 0.875 with 2000 features.


# Feature Selection(Frequency) and accuracy

In [18]:
X_test=[]
Y_test=[]
for instance in Data_New_Test:
    vector_instance=get_vector_text(frequency_best_vocabulary,instance[0])
    X_test.append(vector_instance)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
Y_test_gold=np.asarray(Y_test)


In [19]:
X_train=[]
Y_train=[]
for instance in Data_New_Train:
    vector_instance=get_vector_text(frequency_best_vocabulary,instance[0])
    X_train.append(vector_instance)
    Y_train.append(instance[1])
    
X_new_train = np.asarray(X_train)
Y_new_train = np.asarray(Y_train)
    
fs_data=SelectKBest(chi2, k=1000).fit(X_new_train, Y_new_train)
X_train_fs = fs_data.transform(X_new_train)
    
print ("Size original training matrix: "+str(X_new_train.shape))
print ("Size new training matrix: "+str(X_train_fs.shape))
    
frequency_svm_clf_fs=sklearn.svm.SVC(kernel="linear",gamma='auto') 
frequency_svm_clf_fs.fit(X_train_fs,Y_new_train)

Size original training matrix: (10314, 2000)
Size new training matrix: (10314, 1000)


SVC(gamma='auto', kernel='linear')

In [20]:
Y_text_predictions = frequency_svm_clf_fs.predict(fs_data.transform(X_test))

In [22]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

precision=precision_score(Y_test_gold, Y_text_predictions, average='macro')
recall=recall_score(Y_test_gold, Y_text_predictions, average='macro')
f1=f1_score(Y_test_gold, Y_text_predictions, average='macro')
accuracy=accuracy_score(Y_test_gold, Y_text_predictions)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3)))

Precision: 0.875
Recall: 0.87
F1-Score: 0.872
Accuracy: 0.873


In [21]:
from sklearn.metrics import classification_report
# Y_text_predictions=svm_clf.predict(X_test)
print(classification_report(Y_test_gold, Y_text_predictions))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86       260
           1       0.88      0.83      0.85       222
           2       0.89      0.86      0.87       252
           3       0.83      0.93      0.88       296
           4       0.91      0.88      0.89       259

    accuracy                           0.87      1289
   macro avg       0.88      0.87      0.87      1289
weighted avg       0.87      0.87      0.87      1289



# Number of words feature

In [44]:
list_num_features=[1000,2000,5000,10000]
best_accuracy_dev=0.0
for num_features in list_num_features:
  # First, we get the vocabulary from the training set and train our svm classifier
    vocabulary=get_num_vocabulary(Data_New_Train, num_features)  
    svm_clf=train_svm_classifier(Data_New_Train, vocabulary)
  # Then, we transform our dev set into vectors and make the prediction on this set
    X_dev=[]
    for instance in Data_New_Dev:
        vector_instance=get_vector_text(vocabulary,instance[0])
        X_dev.append(vector_instance)
    X_dev=np.asarray(X_dev)
    Y_dev_predictions=svm_clf.predict(X_dev)
  # Finally, we get the accuracy results of the classifier
    accuracy_dev=accuracy_score(Y_dev_gold, Y_dev_predictions)
    print ("Accuracy with "+str(num_features)+": "+str(round(accuracy_dev,3)))
    if accuracy_dev>=best_accuracy_dev:
        best_accuracy_dev=accuracy_dev
        best_num_features=num_features
        num_best_vocabulary=vocabulary
        num_best_svm_clf=svm_clf
print ("\n Number of words vocabulary,best accuracy overall in the dev set is "+str(round(best_accuracy_dev,3))+" with "+str(best_num_features)+" features.")

Accuracy with 1000: 0.329
Accuracy with 2000: 0.396
Accuracy with 5000: 0.552
Accuracy with 10000: 0.713

 Number of words vocabulary,best accuracy overall in the dev set is 0.713 with 10000 features.


# Feature Selection(Number of words) and accuracy

In [46]:
X_test=[]
Y_test=[]
for instance in Data_New_Test:
    vector_instance=get_vector_text(num_best_vocabulary,instance[0])
    X_test.append(vector_instance)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
Y_test_gold=np.asarray(Y_test)

In [47]:
X_train=[]
Y_train=[]
for instance in Data_New_Train:
    vector_instance=get_vector_text(num_best_vocabulary,instance[0])
    X_train.append(vector_instance)
    Y_train.append(instance[1])
    
X_new_train = np.asarray(X_train)
Y_new_train = np.asarray(Y_train)
    
fs_data=SelectKBest(chi2, k=1000).fit(X_new_train, Y_new_train)
X_train_fs = fs_data.transform(X_new_train)
    #X_train_new = SelectKBest(chi2, k=500).fit_transform(X_train, Y_train)
print ("Size original training matrix: "+str(X_new_train.shape))
print ("Size new training matrix: "+str(X_train_fs.shape))
    
num_svm_clf_fs=sklearn.svm.SVC(kernel="linear",gamma='auto') # Change the name here, e.g. 'new sentanalysis_svm_clf', and below if you don't want to replace your old classifier.
num_svm_clf_fs.fit(X_train_fs,Y_new_train)

Size original training matrix: (10314, 10000)
Size new training matrix: (10314, 1000)


SVC(gamma='auto', kernel='linear')

In [48]:
Y_text_predictions = num_svm_clf_fs.predict(fs_data.transform(X_test))
print(classification_report(Y_test_gold, Y_text_predictions))

              precision    recall  f1-score   support

           0       0.74      0.58      0.65       260
           1       0.78      0.59      0.67       222
           2       0.82      0.65      0.73       252
           3       0.54      0.94      0.69       296
           4       0.82      0.66      0.73       259

    accuracy                           0.69      1289
   macro avg       0.74      0.68      0.69      1289
weighted avg       0.73      0.69      0.69      1289



In [49]:
precision=precision_score(Y_test_gold, Y_text_predictions, average='macro')
recall=recall_score(Y_test_gold, Y_text_predictions, average='macro')
f1=f1_score(Y_test_gold, Y_text_predictions, average='macro')
accuracy=accuracy_score(Y_test_gold, Y_text_predictions)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3)))

Precision: 0.742
Recall: 0.683
F1-Score: 0.693
Accuracy: 0.694


# Averange length of words feature

In [28]:
list_num_features=[800,1000,1377]
best_accuracy_dev=0.0
for num_features in list_num_features:
  # First, we get the vocabulary from the training set and train our svm classifier
    vocabulary=get_ave_num_vocabulary(Data_New_Train, num_features)  
    svm_clf=train_svm_classifier(Data_New_Train, vocabulary)
  # Then, we transform our dev set into vectors and make the prediction on this set
    X_dev=[]
    for instance in Data_New_Dev:
        vector_instance=get_vector_text(vocabulary,instance[0])
        X_dev.append(vector_instance)
    X_dev=np.asarray(X_dev)
    Y_dev_predictions=svm_clf.predict(X_dev)
  # Finally, we get the accuracy results of the classifier
    accuracy_dev=accuracy_score(Y_dev_gold, Y_dev_predictions)
    print ("Accuracy with "+str(num_features)+": "+str(round(accuracy_dev,3)))
    if accuracy_dev>=best_accuracy_dev:
        best_accuracy_dev=accuracy_dev
        best_num_features=num_features
        ave_num_best_vocabulary=vocabulary
        ave_num_best_svm_clf=svm_clf
print ("\n Averange length of words vocabulary,best accuracy overall in the dev set is "+str(round(best_accuracy_dev,3))+" with "+str(best_num_features)+" features.")

Accuracy with 800: 0.464
Accuracy with 1000: 0.486
Accuracy with 1377: 0.55

 Averange length of words vocabulary,best accuracy overall in the dev set is 0.55 with 1377 features.


# Feature Selection(Averange length of words) and accuracy

In [29]:
X_test=[]
Y_test=[]
for instance in Data_New_Test:
    vector_instance=get_vector_text(ave_num_best_vocabulary,instance[0])
    X_test.append(vector_instance)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
Y_test_gold=np.asarray(Y_test)

In [34]:
X_train=[]
Y_train=[]
for instance in Data_New_Train:
    vector_instance=get_vector_text(ave_num_best_vocabulary,instance[0])
    X_train.append(vector_instance)
    Y_train.append(instance[1])
    
X_new_train = np.asarray(X_train)
Y_new_train = np.asarray(Y_train)
    
fs_data=SelectKBest(chi2, k=650).fit(X_new_train, Y_new_train)
X_train_fs = fs_data.transform(X_new_train)
    #X_train_new = SelectKBest(chi2, k=500).fit_transform(X_train, Y_train)
print ("Size original training matrix: "+str(X_new_train.shape))
print ("Size new training matrix: "+str(X_train_fs.shape))
    
ave_num_svm_clf_fs=sklearn.svm.SVC(kernel="linear",gamma='auto') # Change the name here, e.g. 'new sentanalysis_svm_clf', and below if you don't want to replace your old classifier.
ave_num_svm_clf_fs.fit(X_train_fs,Y_new_train)

Size original training matrix: (10314, 1377)
Size new training matrix: (10314, 650)


SVC(gamma='auto', kernel='linear')

In [35]:
Y_text_predictions = ave_num_svm_clf_fs.predict(fs_data.transform(X_test))
print(classification_report(Y_test_gold, Y_text_predictions))

              precision    recall  f1-score   support

           0       0.64      0.44      0.52       260
           1       0.62      0.36      0.46       222
           2       0.50      0.42      0.46       252
           3       0.40      0.84      0.55       296
           4       0.76      0.46      0.58       259

    accuracy                           0.52      1289
   macro avg       0.59      0.50      0.51      1289
weighted avg       0.58      0.52      0.51      1289



In [36]:
precision=precision_score(Y_test_gold, Y_text_predictions, average='macro')
recall=recall_score(Y_test_gold, Y_text_predictions, average='macro')
f1=f1_score(Y_test_gold, Y_text_predictions, average='macro')
accuracy=accuracy_score(Y_test_gold, Y_text_predictions)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3)))

Precision: 0.586
Recall: 0.504
F1-Score: 0.511
Accuracy: 0.518
